# Visualize User Behavior with RAPIDS TSNE and Item Matrix Factorization
In this notebook, we will use RAPIDS TSNE together with item embeddings from item matrix factorization (thank you [CPMP][1] and [Radek][2] for your notebooks) to visualize users' activity behavior.

In Kaggle's Otto Recommender System competition, the item ids are anonymized. So we do not know which each item id refers to. However, with item matrix factorization we can convert the anonymus item ids into meaningful embeddings. Then similar embeddings will be similar items. If we project the embeddings into the 2D plane (with TSNE, UMAP, PCA etc) and plot them, we can see clusters of similar items. Then one cluster may be clothing and another cluster could be electronics, etc etc.

Using the 2D plane of item embeddings, we can draw dots and lines showing the progression of each users' activity. This allows us to understand how users shop. Do users browse similar items, then view different items, then return to buy the original items. Do users mainly explore similar items? Or do users explore all sorts of items? The plots below will illuminate user behavior and help us understand the data and build better features and models!

**NOTE** The code for this notebook's item matrix factorization comes from CPMP's notebook [here][1] and Radek's notebook [here][2]. If you find my notebook helpful, please upvote their notebooks too!

Enjoy!

[1]: https://www.kaggle.com/code/cpmpml/matrix-factorization-with-gpu
[2]: https://www.kaggle.com/code/radek1/matrix-factorization-pytorch-merlin-dataloader

# Data Preprocessing
We will load and process data with RAPIDS cuDF

In [1]:
import cudf
import pandas as pd
from utils.data.load_data import cache_data_to_cpu
import numpy as np
from pathlib import Path
import gc

train = cudf.read_parquet('../data/otto-full-optimized-memory-footprint/train.parquet')
test = cudf.read_parquet('../data/otto-full-optimized-memory-footprint/test.parquet')

In [2]:
unique_aids = list(train.aid.unique().to_numpy()) + list(test.aid.unique().to_numpy())
unique_aids = np.sort(np.unique(unique_aids))

print(f'Train Unique Aids: {train.aid.nunique():,}')
print(f'Test Unique Aids: {test.aid.nunique():,}')
print(f'Unique Aids: {len(unique_aids):,}')

Train Unique Aids: 1,855,603
Test Unique Aids: 783,486
Unique Aids: 1,855,603


In [3]:
train_pairs = cudf.concat([train, test])[['session', 'aid']]
del train, test
_ = gc.collect()

print(f'train_pairs shape: ({train_pairs.shape[0]:,}, {train_pairs.shape[1]:,})')

train_pairs['aid_next'] = train_pairs.groupby('session').aid.shift(-1)
train_pairs = train_pairs[['aid', 'aid_next']].dropna().reset_index(drop=True)

cardinality_aids = max(train_pairs['aid'].max(), train_pairs['aid_next'].max())
print(f'Cardinality of items is {cardinality_aids:,}')
print(f'train_pairs shape: ({train_pairs.shape[0]:,}, {train_pairs.shape[1]:,})')

train_pairs shape: (223,644,219, 2)
Cardinality of items is 1,855,602
train_pairs shape: (209,072,637, 2)


# Install Merlin Dataloader!
We will feed our PyTorch model with Merlin dataloader!

In [4]:
from merlin.loader.torch import Loader 

train_pairs[:].to_parquet('train_pairs.parquet') # TRAIN WITH ALL DATA
train_pairs[-10_000_000:].to_parquet('valid_pairs.parquet')

from merlin.loader.torch import Loader 
from merlin.io import Dataset

train_ds = Dataset('train_pairs.parquet')
train_dl_merlin = Loader(train_ds, 65536, True)

# Learn Item Embeddings with PyTorch Matrix Factorization Model
We will build a PyTorch model to generate item embeddings via matrix factorization.

In [5]:
import torch
from torch import nn

class MatrixFactorization(nn.Module):
    def __init__(self, n_aids, n_factors):
        super().__init__()
        self.aid_factors = nn.Embedding(n_aids, n_factors, sparse=True)
        
    def forward(self, aid1, aid2):
        aid1 = self.aid_factors(aid1)
        aid2 = self.aid_factors(aid2)
        
        return (aid1 * aid2).sum(dim=1)
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

valid_ds = Dataset('valid_pairs.parquet')
valid_dl_merlin = Loader(valid_ds, 65536, True)

In [6]:
from torch.optim import SparseAdam

num_epochs = 10
lr=0.1

model = MatrixFactorization(cardinality_aids + 1, 32)
optimizer = SparseAdam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

model.to('cuda')
for epoch in range(num_epochs):
    for batch, _ in train_dl_merlin:
        model.train()
        losses = AverageMeter('Loss', ':.4e')
            
        aid1, aid2 = batch['aid'], batch['aid_next']
        aid1 = aid1.to('cuda')
        aid2 = aid2.to('cuda')
        output_pos = model(aid1, aid2)
        output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
        
        output = torch.cat([output_pos, output_neg])
        targets = torch.cat([torch.ones_like(output_pos), torch.zeros_like(output_pos)])
        loss = criterion(output, targets)
        losses.update(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    
    with torch.no_grad():
        accuracy = AverageMeter('accuracy')
        for batch, _ in valid_dl_merlin:
            aid1, aid2 = batch['aid'], batch['aid_next']
            output_pos = model(aid1, aid2)
            output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
            accuracy_batch = torch.cat([output_pos.sigmoid() > 0.5, output_neg.sigmoid() < 0.5]).float().mean()
            accuracy.update(accuracy_batch, aid1.shape[0])
            
    print(f'{epoch+1:02d}: * TrainLoss {losses.avg:.3f}  * Accuracy {accuracy.avg:.3f}')

01: * TrainLoss 0.600  * Accuracy 0.719
02: * TrainLoss 0.597  * Accuracy 0.725
03: * TrainLoss 0.592  * Accuracy 0.727
04: * TrainLoss 0.593  * Accuracy 0.728
05: * TrainLoss 0.589  * Accuracy 0.729
06: * TrainLoss 0.594  * Accuracy 0.730
07: * TrainLoss 0.590  * Accuracy 0.730
08: * TrainLoss 0.588  * Accuracy 0.730
09: * TrainLoss 0.588  * Accuracy 0.730
10: * TrainLoss 0.591  * Accuracy 0.730


# Extract Item Embeddings
We extract item embeddings from our model's embedding table.

In [7]:
# EXTRACT EMBEDDINGS FROM MODEL EMBEDDING TABLE
embeddings = model.aid_factors.weight.detach().cpu().numpy()
print('Item Matrix Factorization embeddings have shape', embeddings.shape)

Item Matrix Factorization embeddings have shape (1855603, 32)


In [8]:
aids = np.arange(0, len(embeddings))
aids = np.expand_dims(aids, 1)
data = np.concatenate((aids, embeddings), axis=1)
col0 = ['aid']
col1 = [f'aid_embeddings_{i}'for i in range(embeddings.shape[1])]
cols = col0 + col1

aid_df = pd.DataFrame(data, columns=cols)

save_base = Path('../output/features')
file_path = save_base / APPROACH / 'aid' / 'embeddings.parquet'
aid_df.to_parquet(file_path)

aid_df.head()

,aid,aid_embeddings_0,aid_embeddings_1,aid_embeddings_2,aid_embeddings_3,aid_embeddings_4,aid_embeddings_5,aid_embeddings_6,aid_embeddings_7,aid_embeddings_8,...,aid_embeddings_22,aid_embeddings_23,aid_embeddings_24,aid_embeddings_25,aid_embeddings_26,aid_embeddings_27,aid_embeddings_28,aid_embeddings_29,aid_embeddings_30,aid_embeddings_31
0,0.0,-0.756765,-0.876506,0.797957,0.833474,0.807698,1.459579,-0.880050,-0.887415,0.779998,...,-0.927014,-0.963710,0.919051,-0.861982,-0.836275,-0.881735,1.078397,-1.046697,0.902699,-0.761971
1,1.0,0.997984,0.976318,-1.001223,-1.041720,-1.304604,1.402810,0.982275,1.342228,1.303908,...,1.074208,0.915103,-1.197821,0.892385,1.156410,1.011524,-1.128922,-0.778731,1.263883,1.062263
2,2.0,1.215585,1.456276,-1.368885,-1.362887,-1.149678,1.348378,1.392685,1.484847,-1.232229,...,1.382044,1.361757,-1.569665,1.151968,1.382256,1.188135,-1.176303,1.326288,-1.213133,1.354664
3,3.0,0.742096,0.550203,-0.502212,1.501826,-0.611016,-1.528150,-1.227302,-1.218148,-0.527265,...,0.824047,0.558707,-0.751838,0.621403,0.543416,0.572857,-0.717171,-1.038690,-0.650917,0.671957
4,4.0,1.381064,1.387579,-1.254696,-1.009053,-1.448489,1.262781,1.300559,1.429187,-1.233419,...,1.379731,1.342145,-1.298025,1.205266,1.408336,1.299946,-1.343674,1.397137,-1.204235,1.261737


# Visualize User Behavior with RAPIDS TSNE
We will use RAPIDS cuML's TSNE to reduce embedding dimension to 2 dimensions so we can plot in x-y-plane.

In [9]:
# IMPORT RAPIDS TSNE
from cuml import UMAP, TSNE, PCA
import matplotlib.pyplot as plt, numpy as np
import matplotlib.patches as mpatches, cuml
print('RAPIDS cuML version',cuml.__version__)

# FIT TRANSFORM TSNE
em_pca = PCA(n_components=5).fit_transform(embeddings)
print(f'PCA embeddings have shape ({em_pca.shape[0]:,} , {em_pca.shape[1]:,})')

RAPIDS cuML version 22.10.01
PCA embeddings have shape (1,855,603 , 5)


In [10]:
aids = np.arange(0, len(em_pca))
aids = np.expand_dims(aids, 1)
data = np.concatenate((aids, em_pca), axis=1)
col0 = ['aid']
col1 = [f'aid_pca_{i}'for i in range(em_pca.shape[1])]
cols = col0 + col1

aid_df = pd.DataFrame(data, columns=cols)

save_base = Path('../output/features')
file_path = save_base / APPROACH / 'aid' / 'pca.parquet'
aid_df.to_parquet(file_path)
aid_df.head()

,aid,aid_pca_0,aid_pca_1,aid_pca_2,aid_pca_3,aid_pca_4
0,0.0,-3.721172,2.253680,-0.885174,-1.734350,0.063166
1,1.0,4.849959,0.447388,1.622867,-1.963760,-0.722287
2,2.0,7.365570,-0.392378,-1.179907,-0.029992,0.204720
3,3.0,1.196136,-0.918033,0.665705,2.070251,-1.878428
4,4.0,7.482162,-0.747572,-1.113639,-0.130242,0.151078


In [11]:
# IMPORT RAPIDS TSNE
from cuml import UMAP, TSNE, PCA
import matplotlib.pyplot as plt, numpy as np
import matplotlib.patches as mpatches, cuml
print('RAPIDS cuML version',cuml.__version__)

# FIT TRANSFORM TSNE
em_tsne = TSNE(n_components=2).fit_transform(embeddings)
print(f'TSNE embeddings have shape ({em_tsne.shape[0]:,} , {em_tsne.shape[1]:,})')

RAPIDS cuML version 22.10.01
[W] [19:00:06.476063] # of Nearest Neighbors should be at least 3 * perplexity. Your results might be a bit strange...


/home/mdunlap/anaconda3/envs/kaggle-sysrec/lib/python3.9/site-packages/cuml/internals/api_decorators.py:794: UserWarning: Starting from version 22.04, the default method of TSNE is 'fft'.
  return func(**kwargs)


TSNE embeddings have shape (1,855,603 , 2)


In [12]:
aids = np.arange(0, len(em_tsne))
aids = np.expand_dims(aids, 1)
data = np.concatenate((aids, em_tsne), axis=1)
col0 = ['aid']
col1 = [f'aid_tsne_{i}'for i in range(em_tsne.shape[1])]
cols = col0 + col1

aid_df = pd.DataFrame(data, columns=cols)


save_base = Path('../output/features')
aid_df.to_parquet(save_base / 'validation' / 'aid' / 'tsne.parquet')
aid_df.to_parquet(save_base / 'test' / 'aid' / 'tsne.parquet')
aid_df.head()

,aid,aid_tsne_0,aid_tsne_1
0,0.0,-69.710762,6.494036
1,1.0,51.783504,76.079674
2,2.0,66.377762,15.025543
3,3.0,62.954735,11.189374
4,4.0,-29.359709,60.543625
